# Introduction to ipyaladin

In [ ]:
%pip install ipyaladin
%pip install sidecar

In [ ]:
import micropip, http.client, pyodide_http
await micropip.install(["ssl","https://files.pythonhosted.org/packages/c3/c8/d74794dff5c6802994d3dfe506d691d9e0ff3e6b2f34df577899685b2617/astroquery-0.4.7.dev9008-py3-none-any.whl"])
pyodide_http.patch_all()
from importlib import reload
reload(http.client)

In [ ]:
from ipyaladin import Aladin
from sidecar import Sidecar
from astroquery.vizier import Vizier

In [ ]:
sidecar = Sidecar(title='aladin')
aladin = Aladin()
with sidecar:
    display(aladin)

## 1. Let's find the catalog identifier of DESI legacy survey in Vizier

In [ ]:
list_catalogs = Vizier.find_catalogs(["DESI", "legacy"])
[print(catalog, description.description) for catalog, description in list_catalogs.items()];

The identifier is `VII/192`

In [ ]:
catalog = "VII/292"

## 2. A glimpse at its coverage

In [ ]:
catalogue_coverage = MOC.from_fits(
    f"https://cdsarc.cds.unistra.fr/viz-bin/moc/{CATALOGUE}",
)
catalogue_coverage

The identifier is `VII/192`. 

## 2. Let's query for sources from DESI in a cone around a point in the sky

We'll look at how to do that with the Vizier class in astroquery.

In [ ]:
# here, let's take all columns and remove the upper limit on the number of rows
# we comment the line about how to do the query in real python
# table_2Mass = Vizier(columns=["**"], row_limit=-1).query_region("08:29:58, 1:30:10", 10 * u.arcmin, catalog="II/246")
# and get it from the file system
table_2Mass = parse_single_table("data/2MASS_PSC.vot").to_table()
table_2Mass

## 3. get it into ipyaladin

We load the widget around the desired position first. The following cell adds the table to the view.

In [ ]:
aladin = Aladin(fov=2, target="08:29:58, 1:30:10")
aladin

In [ ]:
aladin.add_table(table_2Mass)

In [ ]:
aladin.height=800